<a href="https://colab.research.google.com/github/stayup24h/Hangul-to-Unicode-Obfuscation-Project/blob/main/model_building_convnet_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 자기 계정의 드라이브를 코랩 폴더에 마운트함
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pwd

/content


In [ ]:
!pip install keras

In [ ]:
# initial
import os
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# 데이터 주무르기
import numpy as np
import tensorflow as tf

# numpy tensor 한 개
my_tensor = np.load("/content/drive/MyDrive/datasets/tensor_printed_0.npy")

# tensor 모양 출력
print("Shape of my_tensor:", my_tensor.shape)

# 채널 차원이 없으니 하나 추가 (50000, 150, 150) -> (50000, 150, 150, 1)
my_tensor = np.expand_dims(my_tensor, axis=-1)

# 64 * 64으로 변경 (50000, 150, 150, 1) -> (50000, 64, 64, 1)
my_tensor = np.array([tf.image.resize(img, (64, 64)) for img in my_tensor])

# float 노멀라이징
my_tensor = my_tensor.astype(np.float32) / 255.0

In [ ]:
my_tensor.shape

In [ ]:
# Load the labels
labels = np.load("/content/drive/MyDrive/datasets/tensor_printed_0_labels.npy")

# Print the shape of the labels
print("Shape of labels:", labels.shape)

In [ ]:
# 모델 설계
def model(input_shape):
    input_layer = layers.Input(shape=input_shape)

    x = Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape)(input_layer)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)

    cho = layers.Dense(19, activation='softmax', name="cho")(x)
    jung = layers.Dense(21, activation='softmax', name="jung")(x)
    jong = layers.Dense(28, activation='softmax', name="jong")(x)
    model = Model(inputs=input_layer, outputs=[cho, jung, jong])

    return model

In [ ]:
# 모델 체크포인트 관리 클래스

checkpoint_path = "training_checkpoints/epoch_{epoch:04d}/model.weights.h5"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq='epoch'
)

class CheckpointManager:
    def __init__(self, checkpoint_path_template, max_recent=3, extra_interval=50):
        self.checkpoint_path_template = checkpoint_path_template
        self.max_recent = max_recent
        self.extra_interval = extra_interval
        self.recent_epochs = []

    def load(self, model, epoch, checkpoint_path_template=checkpoint_path):
        path = checkpoint_path_template.format(epoch=epoch)
        if os.path.exists(path):
            model.load_weights(path)
            print(f"Loaded weights from {path}")
            return True
        else:
            print(f"No checkpoint found at {path}")
            return False

    def save(self, model, epoch):
        # Always save at extra_interval epochs
        if epoch % self.extra_interval == 0 and epoch not in self.recent_epochs:
            path = self.checkpoint_path_template.format(epoch=epoch)
            model.save_weights(path)
            print(f"Extra checkpoint saved for epoch {epoch} at {path}")

        # Save recent checkpoints
        self.recent_epochs.append(epoch)
        if len(self.recent_epochs) > self.max_recent:
            # Remove oldest checkpoint from recent
            old_epoch = self.recent_epochs.pop(0)
            old_path = self.checkpoint_path_template.format(epoch=old_epoch)
            if os.path.exists(old_path):
                os.remove(old_path)
                print(f"Removed old checkpoint at {old_path}")

        path = self.checkpoint_path_template.format(epoch=epoch)
        model.save_weights(path)
        print(f"Checkpoint saved for epoch {epoch} at {path}")

In [ ]:
from tensorflow.keras.optimizers import Adam

the_model = model((64, 64, 1))
optimizer = Adam(learning_rate=0.0001)
the_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', 'accuracy', 'accuracy'])
the_model.summary()

In [ ]:
cho_labels = labels[:, :19]
jung_labels = labels[:, 19:19+21]
jong_labels = labels[:, 19+21:]

the_model.fit(my_tensor, [cho_labels, jung_labels, jong_labels], epochs=10)

In [ ]:
# Make predictions on a small subset of the data
predictions = the_model.predict(my_tensor[:10])

# The predictions will be a list of arrays, one for each output (cho, jung, jong)
cho_predictions = predictions[0]
jung_predictions = predictions[1]
jong_predictions = predictions[2]

# Print the predictions
print("Cho predictions:\n", cho_predictions)
print("\nJung predictions:\n", jung_predictions)
print("\nJong predictions:\n", jong_predictions)